# Анализ рынка игровой индустрии и вгровых платформ в период с 2000 по 2013 год

- Автор: Матюнин Петр
- Дата: 11.12.2024

## Цели и задачи проекта ##

Произвести обработку данных и произвести их анализ для последующего написания статьи о развитии игровой индустрии в начала XXI века.  
    
1) Сделать обзор игровых платформ
2) Изучить объемы продаж игр разных жанров и региональные предпочтения игроков.


## Описание данных


В качестве базы для исследования, нам представлены исторические данные о продажах игр, сделанных в разных жанрах и выпущенных на разных платформах, а также пользовательские и экспертные оценки игр. Срез данных представлен за 1980 - 2016 года, однако, по запросу заказчика, анализ необходимо произвести на основе данных за 2000 - 2013 года.


## Содержимое проекта

1. Цели и задачи проекта
2. Описание данных
3. Содержимое проекта
4. Загрузка данных и знакомство с ними
5. Проверка ошибок в данных и их предобработка
    1. Типы данных
    2. Наличие пропусков в данных
    3. Явные и неявные дубликаты в данных
6. Фильтрация данных
7. Категоризация данных
8. Итоговый вывод

   

---

## Загрузка данных и знакомство с ними

Прежде чем производить анализ данных, подключим необходимые, для работы, библиотеки а так же получим данные и проверим насколько они соответствуют заявленным ожиданиям

In [1]:
# используемые библиотеки
import pandas as pd
import numpy as np

In [2]:
# получение данных и формирование первичного датасета 
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [3]:
display(df.head())

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
# за какие года прдставлены данные
min_release_year = df['Year of Release'].min()
max_release_year = df['Year of Release'].max()
print(f'Данные представлены с {min_release_year} по {max_release_year} года')

Данные представлены с 1980.0 по 2016.0 года


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [6]:
display(df.head())

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [7]:
display(df.sample(3))

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
5332,LocoRoco 2,PSP,2008.0,Platform,0.21,0.09,0.01,0.04,85.0,8.4,E
5737,Test Drive Unlimited,X360,2006.0,Racing,0.23,0.04,0.02,0.02,82.0,8,E10+
6763,Hybrid Heaven,N64,1999.0,Role-Playing,0.16,0.04,0.05,0.00,NaN,NaN,NaN


In [8]:
print('Поля и относительная доля пропусков в них')
display(df.isna().mean().to_frame().sort_values(by=0, ascending=False).style.background_gradient(cmap='coolwarm', axis=0).format("{:.2f}"))

,0
Critic Score,0.51
Rating,0.41
User Score,0.40
Year of Release,0.02
Name,0.00
Genre,0.00
Platform,0.00
NA sales,0.00
EU sales,0.00
JP sales,0.00


По первому взгляду на полученные данные мы можем составить первоначальное мнение. Датасет содержит **11 столбцов** и **16 956 строк**
данных. Из них **4 столбца** типа **float** и **7 столбцов** типа **object**.   
Обратив внимание на типы данных, можно заметить что не все столбцы
содержат удобный, для работы, формат данных. Таким образом необходимо поменять форматы столбцов год релиза (*Year of Release*),
продажи в Европе (*EU sales*), продажи в Японии (*JP sales*), оценки пользователей (*User Score*). Наименования столбцов необходимо
преобразовать в рабучую форму типа *snake case*.   
    **Данные представлены за 1980 по 2016 года**, тогда как анализ *необходимо провести* за
**период с 2000 по 2013 года**, а значит необходимо провести фильтрацию данных.   
Так же данные *содержат много пропусков*. Столбец года релиза
содержит всего **2% пропусков**, тогда как столбцы с оценками критиков, пользователей а так же **столбец с рейтингом содержат от 40 до
51% пропусков**, что является крайне большим показателем.

---

## Проверка ошибок в данных и их предобработка



Прежде чем приступать к анализу, нам так же необходимо произвести предъобработку данных

In [9]:
# Пропуски в абсолютных значениях
display(df.isna().sum())

Name                  2
Platform              0
Year of Release     275
Genre                 2
NA sales              0
EU sales              0
JP sales              0
Other sales           0
Critic Score       8714
User Score         6804
Rating             6871
dtype: int64

In [10]:
# Пропуски в долевых значениях
display(round(df.isna().mean(), 2))

Name               0.00
Platform           0.00
Year of Release    0.02
Genre              0.00
NA sales           0.00
EU sales           0.00
JP sales           0.00
Other sales        0.00
Critic Score       0.51
User Score         0.40
Rating             0.41
dtype: float64

In [11]:
# Названия полей
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [12]:
# Приведение названий полей к единому формату
df.columns = [x.lower().replace(' ', '_') for x in df.columns.values]
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

In [13]:
df.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8714
user_score         6804
rating             6871
dtype: int64

### Типы данных


In [14]:
#Находим текстовые значения в данных. Таким же образом ищем текстовые значения в других столбцах и следующим шагом удаляем их
display(df['jp_sales'].sort_values().unique())

array(['0.0', '0.01', '0.02', '0.03', '0.04', '0.05', '0.06', '0.07',
       '0.08', '0.09', '0.1', '0.11', '0.12', '0.13', '0.14', '0.15',
       '0.16', '0.17', '0.18', '0.19', '0.2', '0.21', '0.22', '0.23',
       '0.24', '0.25', '0.26', '0.27', '0.28', '0.29', '0.3', '0.31',
       '0.32', '0.33', '0.34', '0.35', '0.36', '0.37', '0.38', '0.39',
       '0.4', '0.41', '0.42', '0.43', '0.44', '0.45', '0.46', '0.47',
       '0.48', '0.49', '0.5', '0.51', '0.52', '0.53', '0.54', '0.55',
       '0.56', '0.57', '0.58', '0.59', '0.6', '0.61', '0.62', '0.63',
       '0.64', '0.65', '0.66', '0.67', '0.68', '0.69', '0.7', '0.71',
       '0.72', '0.73', '0.74', '0.75', '0.76', '0.77', '0.78', '0.79',
       '0.8', '0.81', '0.82', '0.83', '0.84', '0.85', '0.86', '0.87',
       '0.88', '0.89', '0.9', '0.91', '0.92', '0.93', '0.94', '0.95',
       '0.96', '0.97', '0.98', '0.99', '1.0', '1.01', '1.02', '1.03',
       '1.04', '1.05', '1.06', '1.07', '1.08', '1.09', '1.1', '1.11',
       '1.12', '1.

In [15]:
df = df.dropna(subset=['name', 'platform', 'eu_sales', 'jp_sales', 'year_of_release'])
display(df.isna().sum())

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8594
user_score         6705
rating             6778
dtype: int64

In [16]:
display(df['eu_sales'].sort_values().unique())

array(['0.0', '0.01', '0.02', '0.03', '0.04', '0.05', '0.06', '0.07',
       '0.08', '0.09', '0.1', '0.11', '0.12', '0.13', '0.14', '0.15',
       '0.16', '0.17', '0.18', '0.19', '0.2', '0.21', '0.22', '0.23',
       '0.24', '0.25', '0.26', '0.27', '0.28', '0.29', '0.3', '0.31',
       '0.32', '0.33', '0.34', '0.35', '0.36', '0.37', '0.38', '0.39',
       '0.4', '0.41', '0.42', '0.43', '0.44', '0.45', '0.46', '0.47',
       '0.48', '0.49', '0.5', '0.51', '0.52', '0.53', '0.54', '0.55',
       '0.56', '0.57', '0.58', '0.59', '0.6', '0.61', '0.62', '0.63',
       '0.64', '0.65', '0.66', '0.67', '0.68', '0.69', '0.7', '0.71',
       '0.72', '0.73', '0.74', '0.75', '0.76', '0.77', '0.78', '0.79',
       '0.8', '0.81', '0.82', '0.83', '0.84', '0.85', '0.86', '0.87',
       '0.88', '0.89', '0.9', '0.91', '0.92', '0.93', '0.94', '0.95',
       '0.96', '0.97', '0.98', '0.99', '1.0', '1.01', '1.02', '1.03',
       '1.04', '1.05', '1.06', '1.07', '1.08', '1.09', '1.1', '1.11',
       '1.12', '1.

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16679 non-null  object 
 1   platform         16679 non-null  object 
 2   year_of_release  16679 non-null  float64
 3   genre            16679 non-null  object 
 4   na_sales         16679 non-null  float64
 5   eu_sales         16679 non-null  object 
 6   jp_sales         16679 non-null  object 
 7   other_sales      16679 non-null  float64
 8   critic_score     8085 non-null   float64
 9   user_score       9974 non-null   object 
 10  rating           9901 non-null   object 
dtypes: float64(4), object(7)
memory usage: 1.5+ MB


In [18]:
#Приводим столбец critic_score к типу данных int
df[['year_of_release', 'critic_score']] = df[['year_of_release', 'critic_score']].astype('Int64')

In [19]:
#Заменяем текстовые значения на пропуски и заменяем типы данных
df['eu_sales'] = df['eu_sales'].replace('unknown', np.nan)
df['jp_sales'] = df['jp_sales'].replace('unknown', np.nan)
df['user_score'] = df['user_score'].replace('tbd', np.nan)
df[['eu_sales', 'jp_sales', 'user_score']] = df[['eu_sales', 'jp_sales', 'user_score']].astype('float64')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16679 non-null  object 
 1   platform         16679 non-null  object 
 2   year_of_release  16679 non-null  Int64  
 3   genre            16679 non-null  object 
 4   na_sales         16679 non-null  float64
 5   eu_sales         16673 non-null  float64
 6   jp_sales         16675 non-null  float64
 7   other_sales      16679 non-null  float64
 8   critic_score     8085 non-null   Int64  
 9   user_score       7558 non-null   float64
 10  rating           9901 non-null   object 
dtypes: Int64(2), float64(5), object(4)
memory usage: 1.6+ MB


In [20]:
def sales_filling(x):
    if x['eu_sales'] == np.nan:
        return df['eu_sales'].mean()
    else:
        return x['eu_sales']
df['eu_sales'] = df.apply(sales_filling, axis=1)

In [21]:
display(df['eu_sales'].sort_values().unique())

array([0.000e+00, 1.000e-02, 2.000e-02, 3.000e-02, 4.000e-02, 5.000e-02,
       6.000e-02, 7.000e-02, 8.000e-02, 9.000e-02, 1.000e-01, 1.100e-01,
       1.200e-01, 1.300e-01, 1.400e-01, 1.500e-01, 1.600e-01, 1.700e-01,
       1.800e-01, 1.900e-01, 2.000e-01, 2.100e-01, 2.200e-01, 2.300e-01,
       2.400e-01, 2.500e-01, 2.600e-01, 2.700e-01, 2.800e-01, 2.900e-01,
       3.000e-01, 3.100e-01, 3.200e-01, 3.300e-01, 3.400e-01, 3.500e-01,
       3.600e-01, 3.700e-01, 3.800e-01, 3.900e-01, 4.000e-01, 4.100e-01,
       4.200e-01, 4.300e-01, 4.400e-01, 4.500e-01, 4.600e-01, 4.700e-01,
       4.800e-01, 4.900e-01, 5.000e-01, 5.100e-01, 5.200e-01, 5.300e-01,
       5.400e-01, 5.500e-01, 5.600e-01, 5.700e-01, 5.800e-01, 5.900e-01,
       6.000e-01, 6.100e-01, 6.200e-01, 6.300e-01, 6.400e-01, 6.500e-01,
       6.600e-01, 6.700e-01, 6.800e-01, 6.900e-01, 7.000e-01, 7.100e-01,
       7.200e-01, 7.300e-01, 7.400e-01, 7.500e-01, 7.600e-01, 7.700e-01,
       7.800e-01, 7.900e-01, 8.000e-01, 8.100e-01, 

In [22]:
df[['eu_sales', 'jp_sales', 'user_score']] = df[['eu_sales', 'jp_sales', 'user_score']].apply(pd.to_numeric, errors='coerce')

Мы произвели проверку полей датасета на соответствие форматов и привели все поля к необходимому, для анализа, формату данных.

### Наличие пропусков в данных


На данном этапе мы имеем   
* **8594 пропусков** с оценками критиков (critic_score), 
* **9121 пропуск** с оценками пользователей (user_score),
* **6778 пропусков** с рейтингом.
  
Данные были собраны из открытых источников. Может быть много причин пропусков. Вероятно они могут зависеть от того насколько популярна игра или от того на какой площадке она продавалась. Пропуски в рейтинге могут быть обусловлены тем, что в разных странах играм может быть присвоен разный рейтинг. Оценки пользователей так же могут зависеть от платформы,  на которой выпускалась игра.

In [23]:
# Показатель user_score увеличился после преобразования tbd в nan
display(df.isna().sum())

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8594
user_score         9121
rating             6778
dtype: int64

In [24]:
display(round(df.isna().mean(), 2))

name               0.00
platform           0.00
year_of_release    0.00
genre              0.00
na_sales           0.00
eu_sales           0.00
jp_sales           0.00
other_sales        0.00
critic_score       0.52
user_score         0.55
rating             0.41
dtype: float64

Данные содержат всего **6 пропусков** в столбце *eu_sales* и **4 пропуска** в столбце *jp_sales*. На общем фоне данных эти показатели *не являются значительными* и практически не окажут влияния на итоговый анализ.
У нас нет источников для восполнения данных по оценкам игр, а заменять половину данных средними значениями не имеет смысла, так как это не даст объективных значений при анализе, а значит данные значения можно оставить без изменения.

### Явные и неявные дубликаты в данных


После приведения полей к необходимым типам и обработки пропусков в данных, нам необходимо проверить данные на дубликаты.

In [25]:
# Выводим уникальные значения, для определения на наличие скрытых дубликатов
display(df['platform'].sort_values().unique())

array(['2600', '3DO', '3DS', 'DC', 'DS', 'GB', 'GBA', 'GC', 'GEN', 'GG',
       'N64', 'NES', 'NG', 'PC', 'PCFX', 'PS', 'PS2', 'PS3', 'PS4', 'PSP',
       'PSV', 'SAT', 'SCD', 'SNES', 'TG16', 'WS', 'Wii', 'WiiU', 'X360',
       'XB', 'XOne'], dtype=object)

In [26]:
display(df['year_of_release'].sort_values().unique())

<IntegerArray>
[1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Length: 37, dtype: Int64

In [27]:
display(df['genre'].sort_values().unique())

array(['ACTION', 'ADVENTURE', 'Action', 'Adventure', 'FIGHTING',
       'Fighting', 'MISC', 'Misc', 'PLATFORM', 'PUZZLE', 'Platform',
       'Puzzle', 'RACING', 'ROLE-PLAYING', 'Racing', 'Role-Playing',
       'SHOOTER', 'SIMULATION', 'SPORTS', 'STRATEGY', 'Shooter',
       'Simulation', 'Sports', 'Strategy'], dtype=object)

In [28]:
display(df['rating'].sort_values().unique())

array(['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T', nan], dtype=object)

In [29]:
df['name'] = df['name'].str.lower()
df['platform'] = df['platform'].str.upper()
df['genre'] = df['genre'].str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16679 non-null  object 
 1   platform         16679 non-null  object 
 2   year_of_release  16679 non-null  Int64  
 3   genre            16679 non-null  object 
 4   na_sales         16679 non-null  float64
 5   eu_sales         16673 non-null  float64
 6   jp_sales         16675 non-null  float64
 7   other_sales      16679 non-null  float64
 8   critic_score     8085 non-null   Int64  
 9   user_score       7558 non-null   float64
 10  rating           9901 non-null   object 
dtypes: Int64(2), float64(5), object(4)
memory usage: 1.6+ MB


In [30]:
#Подсчет явных дубликатов
display(df.duplicated(['name', 'platform', 'year_of_release', 'genre']).sum())

236

In [31]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16444 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16444 non-null  object 
 1   platform         16444 non-null  object 
 2   year_of_release  16444 non-null  Int64  
 3   genre            16444 non-null  object 
 4   na_sales         16444 non-null  float64
 5   eu_sales         16438 non-null  float64
 6   jp_sales         16440 non-null  float64
 7   other_sales      16444 non-null  float64
 8   critic_score     7983 non-null   Int64  
 9   user_score       7463 non-null   float64
 10  rating           9768 non-null   object 
dtypes: Int64(2), float64(5), object(4)
memory usage: 1.5+ MB


По результатам предобработки данных было выявлено и удалено **277 пропусков** в данных, а так же **236 дубликатов**.  
Количество дубликатов и пропусков *не имело критической массы*, в связи с чем данные можно было удалить.

---

## Фильтрация данных


Поскольку исследование необходимо произвети за *период с 2000 по 2013 год*, произведем фильтрацию и сформируем актуальный датафрейм. Так же выведем основную информацию нового датафрейма

In [32]:
df_actual = df.loc[(df['year_of_release'] >=2000) & (df['year_of_release'] <= 2013)].copy()

In [33]:
display(df_actual.head())

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,WII,2006,sports,41.36,28.96,3.77,8.45,76,8.0,E
2,mario kart wii,WII,2008,racing,15.68,12.76,3.79,3.29,82,8.3,E
3,wii sports resort,WII,2009,sports,15.61,10.93,3.28,2.95,80,8.0,E
6,new super mario bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89,8.5,E
7,wii play,WII,2006,misc,13.96,9.18,2.93,2.84,58,6.6,E


In [34]:
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12781 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12781 non-null  object 
 1   platform         12781 non-null  object 
 2   year_of_release  12781 non-null  Int64  
 3   genre            12781 non-null  object 
 4   na_sales         12781 non-null  float64
 5   eu_sales         12776 non-null  float64
 6   jp_sales         12777 non-null  float64
 7   other_sales      12781 non-null  float64
 8   critic_score     7169 non-null   Int64  
 9   user_score       6483 non-null   float64
 10  rating           8723 non-null   object 
dtypes: Int64(2), float64(5), object(4)
memory usage: 1.2+ MB


In [35]:
display(df_actual['year_of_release'].sort_values().unique())

<IntegerArray>
[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
 2013]
Length: 14, dtype: Int64

---

## Категоризация данных
    

Для целей анализа, нам необходимо добавить к первичному датасету несколько полей с категоризацией данных

In [36]:
# Добавляем поле с градацией по оценкам
def user_level(x):
    if x['user_score'] >= 0 and x['user_score'] < 3:
        return 'низкая оценка'
    elif x['user_score'] >= 3 and x['user_score'] < 8:
        return 'средняя оценка'
    elif x['user_score'] >= 8 and x['user_score'] <= 10:
        return 'высокая оценка'
    else:
        return 'нет оценки от пользователей'
    
df_actual['users_level'] = df_actual.apply(user_level, axis=1)
display(df_actual.head(10))

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,users_level
0,wii sports,WII,2006,sports,41.36,28.96,3.77,8.45,76,8.0,E,высокая оценка
2,mario kart wii,WII,2008,racing,15.68,12.76,3.79,3.29,82,8.3,E,высокая оценка
3,wii sports resort,WII,2009,sports,15.61,10.93,3.28,2.95,80,8.0,E,высокая оценка
6,new super mario bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89,8.5,E,высокая оценка
7,wii play,WII,2006,misc,13.96,9.18,2.93,2.84,58,6.6,E,средняя оценка
8,new super mario bros. wii,WII,2009,platform,14.44,6.94,4.70,2.24,87,8.4,E,высокая оценка
10,nintendogs,DS,2005,simulation,9.05,10.95,1.93,2.74,<NA>,NaN,NaN,нет оценки от пользователей
11,mario kart ds,DS,2005,racing,9.71,7.47,4.13,1.90,91,8.6,E,высокая оценка
13,wii fit,WII,2007,sports,8.92,8.03,3.60,2.15,80,7.7,E,средняя оценка
14,kinect adventures!,X360,2010,misc,15.00,4.89,0.24,1.69,61,6.3,E,средняя оценка


In [37]:
def critic_level_maker(x):
    if pd.isna(x['critic_score']):
        return 'нет оценки от пользователей'
    if x['critic_score'] >= 0 and x['critic_score'] < 30:
        return 'низкая оценка'
    elif x['critic_score'] >= 30 and x['critic_score'] < 80:
        return 'средняя оценка'
    elif x['critic_score'] >= 80 and x['critic_score'] <= 100:
        return 'высокая оценка'
    else:
        return 'нет оценки от пользователей'
    
df_actual.loc[:, 'critic_level'] = df_actual.apply(critic_level_maker, axis=1)
display(df_actual.head(10))

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,users_level,critic_level
0,wii sports,WII,2006,sports,41.36,28.96,3.77,8.45,76,8.0,E,высокая оценка,средняя оценка
2,mario kart wii,WII,2008,racing,15.68,12.76,3.79,3.29,82,8.3,E,высокая оценка,высокая оценка
3,wii sports resort,WII,2009,sports,15.61,10.93,3.28,2.95,80,8.0,E,высокая оценка,высокая оценка
6,new super mario bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89,8.5,E,высокая оценка,высокая оценка
7,wii play,WII,2006,misc,13.96,9.18,2.93,2.84,58,6.6,E,средняя оценка,средняя оценка
8,new super mario bros. wii,WII,2009,platform,14.44,6.94,4.70,2.24,87,8.4,E,высокая оценка,высокая оценка
10,nintendogs,DS,2005,simulation,9.05,10.95,1.93,2.74,<NA>,NaN,NaN,нет оценки от пользователей,нет оценки от пользователей
11,mario kart ds,DS,2005,racing,9.71,7.47,4.13,1.90,91,8.6,E,высокая оценка,высокая оценка
13,wii fit,WII,2007,sports,8.92,8.03,3.60,2.15,80,7.7,E,средняя оценка,высокая оценка
14,kinect adventures!,X360,2010,misc,15.00,4.89,0.24,1.69,61,6.3,E,средняя оценка,средняя оценка


In [38]:
users_level_count = df_actual.groupby('users_level')['name'].count()
display(users_level_count.sort_values(ascending=False))

users_level
нет оценки от пользователей    6298
средняя оценка                 4081
высокая оценка                 2286
низкая оценка                   116
Name: name, dtype: int64

In [39]:
critic_level_count = df_actual.groupby('critic_level')['name'].count()
display(critic_level_count.sort_values(ascending=False))

critic_level
нет оценки от пользователей    5612
средняя оценка                 5422
высокая оценка                 1692
низкая оценка                    55
Name: name, dtype: int64

In [40]:
top_platforms = df_actual.groupby('platform')['name'].count()
display(top_platforms.sort_values(ascending=False).head(7))

platform
PS2     2127
DS      2120
WII     1275
PSP     1180
X360    1121
PS3     1087
GBA      811
Name: name, dtype: int64

---

## Итоговый вывод


Начальный датасет содержал:   
* **16956 строк**
* **277 пропусков** в данных важных, для анализа, данных
* **235 дубликатов**
  
Пропуски и дубликаты были удалены из датасета. Так в начальных данных имелись строки с крайне большим количеством пропусков в данных. Данные столбцы не участвовали в итоговой работе с датасетом. Восстановление данных пропусков не было возможным, так как нет источников, откуда брались данных, при необходимости можно сделать запрос заказчику на восстановление данных значений. Замета данных так же не являлась возможной, так как замена на средние значения половины данных, дала бы некорректные значения для половины данных в серии.
Были приобразованы типы данных следующих столбцов
1) *year_of_release*,  
2) *eu_sales*,
3) *jp_sales*,
4) *critic_score*,
5) *user_score*.

Так же были добавлены два новых столбца с категоризацией данных на основе оценок пользователей и критиков.

На основе созданных столбцов была проведена группировка данных, а так же выявлено количество какой категории преобладает в данных.  
По оценкам пользователей и критиков в датасете преобладают игры со средним рейтингом. Однако большая часть данных не содержит оценок вовсе. 
По количеству выпущенных игр, за анализируемый период, лидируют следующие платформы:  
|Платформа| количество|
|:-:|:-|
|PS2|     2 127 игр|
|DS|      2 120 игр|
|WII|     1 275 игр|